In [1]:
from tkinter import ALL
from dash import Dash, html, dcc
import jupyter_dash

from matplotlib.figure import Figure
from matplotlib.pyplot import figure
import plotly.express as px
import plotly.io as pio
pio.templates
from dash.dependencies import Input, Output, State
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from itertools import chain
import pycountry


## Colors

In [2]:
pallete = {
        'Confirmed': '#0091D5',
        'Active': '#DBAE58',
        'Deaths': '#484848',
        'Critical': '#EA6A47',
        'Recovered': '#6AB187',
        'background': '#DADADA'
}

## Test performed map figure

In [3]:
df = pd.read_csv('Data\All_Continents_Union.csv')
df1 = df.iloc[:, [0, 1, 2, 6, 7, 13]]
df2 = df.iloc[:, [0, 7, 2, 6]]
df3 = df.iloc[:, [0, 11, 13]]


Tests_Performed_fig =px.choropleth(df3, locations="iso_alpha",
                                    color="Tests Performed", 
                                    hover_name="Country",
                                    color_continuous_scale=px.colors.sequential.Greens)
Tests_Performed_fig.update_layout(title_text='Tests Performed by Country', title_x=0.45, title_y=0.96, margin={"r":0,"t":45,"l":0,"b":10})

print(" ")

## Vaccination trend line figure

In [4]:
# Global vaccination trend
#_____________________________________________________________________________________________________________________________________#
country_continent = pd.read_csv("Data\Country_and_Continent.csv")
countries_in_each_continent = country_continent.groupby(by='Continent')['Country'].apply(list).reset_index(name='Countries')


vaccination_trend = pd.read_csv("Data\Total_Vaccinations_Trend_Administered_Globally_all.csv").dropna(axis=0, how='any')
indeces_to_be_dropped = vaccination_trend[vaccination_trend.location.isin(['World', 'Upper middle income', 'Lower middle income', 
                                                                            'High income', 'Low income'])].index.to_list()
vaccination_trend = vaccination_trend.drop(index=indeces_to_be_dropped)

continents = sorted(country_continent.Continent.unique().tolist())
Any_country = sorted(list(set(vaccination_trend.location.unique().tolist()) - set(continents)))

Vaccination_trend_Continents = vaccination_trend[vaccination_trend.location.isin(continents)]

Vaccination_trend_Countries = vaccination_trend[vaccination_trend.location.isin(Any_country)]

selected_continent = ['Oceania']
continent = Vaccination_trend_Continents[Vaccination_trend_Continents.location.isin(selected_continent)]
#_____________________________________________________________________________________________________________________________________#

## Vaccination trend map figure

In [5]:
"""Modifications"""
total = []
avg = []
for i in Vaccination_trend_Countries.location:
    vac = Vaccination_trend_Countries[np.isin(Vaccination_trend_Countries.location, i)].total_vaccinations
    total.append(int(vac.values[0]))
    avg.append("{:.2f}".format(np.average(vac)))

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3

codes = [countries.get(country, 'Unknown code') for country in Vaccination_trend_Countries.location]

dic = {'Location':Vaccination_trend_Countries.location, 'Total_Vaccinations':total,'Avg total_vaccinations':avg, 'iso_alpha':codes}
Total_Vaccinations_Trend = pd.DataFrame(dic)

Total_Vaccinations_Trend_fig =px.choropleth(Total_Vaccinations_Trend, locations="iso_alpha",
                                    color="Total_Vaccinations", 
                                    hover_name="Location",
                                    hover_data=["Avg total_vaccinations"],
                                    color_continuous_scale=px.colors.sequential.GnBu)
Total_Vaccinations_Trend_fig.update_layout(title_text='Total Vaccinations Trend Country Wise', title_x=0.5, title_y=0.94, 
                                            margin={"r":10,"t":50,"l":10,"b":0},
                                            coloraxis_colorbar=dict(
                                            title="Vaccination",
                                            thicknessmode="pixels", thickness=10,
                                            lenmode="pixels", len=150,
                                            yanchor="bottom", y=0.14,
                                            xanchor="left", x=0.01
                                        ))
print(" ")

## Global deaths line figure

In [6]:
# Global deaths
#_____________________________________________________________________________________________________________________________________#
global_daily_trend_path = "Data\Global_Deaths_Daily_Trend-all.csv"
global_daily_trend = pd.read_csv(global_daily_trend_path)
global_daily_trend = global_daily_trend[['date', 'location', 'new_deaths', 'new_deaths_smoothed', 'new_cases', 'new_cases_smoothed']].dropna(axis=0, how='any')
indeces_to_be_dropped = global_daily_trend[global_daily_trend.location.isin(['World', 'Upper middle income', 'Lower middle income', 'High income'])].index.to_list()
global_daily_trend = global_daily_trend.drop(index=indeces_to_be_dropped)

deaths_countries = sorted(list(set(global_daily_trend.location.unique().tolist()) - set(continents)))

global_daily_trend_continents = global_daily_trend[global_daily_trend.location.isin(continents)]

global_daily_trend_countries = global_daily_trend[global_daily_trend.location.isin(deaths_countries)]
#_____________________________________________________________________________________________________________________________________#


## Comparison bar Figure 

In [7]:
# Total confirmed vs total recovered vs total deaths
#_____________________________________________________________________________________________________________________________________#
comparison_df_path = "Data\Active_Cases_vs_Recovered_vs_Death_by_Country.csv"
comparison_df = pd.read_csv(comparison_df_path)
indeces_to_be_dropped = comparison_df[comparison_df.Country.isin(['World', 'Upper middle income', 'Lower middle income', 'High income'])].index.to_list()
comparison_df = comparison_df.drop(index=indeces_to_be_dropped)
comparison_df = comparison_df.dropna(axis=0, how='any')

recovered_values_percentage = comparison_df['Total Recovered'] /  comparison_df['Total confirmed']
deaths_values_percentage = comparison_df['Total Deaths'] /  comparison_df['Total confirmed']

comparison_df.insert(2, "Total Recovered percent", recovered_values_percentage)
comparison_df.insert(4, "Total Deaths percent", deaths_values_percentage)
comparison_countries = sorted(comparison_df['Country'].unique().tolist())

selected_countries = ['Afghanistan', 'Albania', 'Algeria', 'Yemen', 'Zimbabwe']
confirmed_selected_countries = comparison_df[comparison_df['Country'].isin(selected_countries)]
confirmed_selected_countries
#_____________________________________________________________________________________________________________________________________#


,Total Active,Total Recovered,Total Recovered percent,Total Deaths,Total Deaths percent,Total confirmed,Country
0,9277.0,162741.0,0.905545,7698.0,0.042834,179716,Afghanistan
1,242.0,272125.0,0.986446,3497.0,0.012677,275864,Albania
2,80593.0,178386.0,0.670992,6875.0,0.025860,265854,Algeria
223,661.0,9009.0,0.762247,2149.0,0.181826,11819,Yemen
225,2115.0,243033.0,0.969642,5494.0,0.021920,250642,Zimbabwe


## Dashboard Layout

In [8]:
app = jupyter_dash.JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# Confirmed
card_content_1 = [
    # 
    dbc.CardBody(
        [
            html.H5("Confirmed Cases", className="card-title1", style={"text-align": "center", 'font-family':'Calibri Light', 'font-size':'150%'}),
            html.P("{:,}".format(int(np.sum(df['Confirmed Cases']))), 
            className="card-text1", style={"text-align": "center", 'font-family':'Calibri Light', 'font-size':'250%'})
        ]
    )
]

# Active
card_content_2 = [
    
    dbc.CardBody(
        [
            html.H5("Active Cases", className="card-title2", style={"text-align": "center", 'font-family':'Calibri Light', 'font-size':'150%'}),
            html.P("{:,}".format(int(np.sum(df['Active Cases']))), 
            className="card-text2", style={"text-align": "center", 'font-family':'Calibri Light', 'font-size':'250%'})
        ]
    )
]

# Deaths
card_content_3 = [
    
    dbc.CardBody(
        [
            html.H5("Deaths", className="card-title3", style={"text-align": "center", 'font-family':'Calibri Light', 'font-size':'150%'}),
            html.P("{:,}".format(int(np.sum(df['Total Deaths']))), 
            className="card-text3", style={"text-align": "center", 'font-family':'Calibri Light', 'font-size':'250%'})
        ]
    )
]

# Critical
card_content_4 = [
    
    dbc.CardBody(
        [
            html.H5("Critical Cases", className="card-title4", style={"text-align": "center", 'font-family':'Calibri Light', 'font-size':'150%'}),
            html.P("{:,}".format(int(np.sum(df['Critical Cases']))), 
            className="card-text4", style={"text-align": "center", 'font-family':'Calibri Light', 'font-size':'250%'})
        ]
    )
]

# Recovered
card_content_5 = [
    
    dbc.CardBody(
        [
            html.H5("Recovered", className="card-title5", style={"text-align": "center", 'font-family':'Calibri Light', 'font-size':'150%'}),
            html.P("{:,}".format(int(np.sum(df['Recovered Cases']))), 
            className="card-text5", style={"text-align": "center", 'font-family':'Calibri Light', 'font-size':'250%'})
        ]
    )
]

# Death rate
card_content_6 = [
    
    dbc.CardBody(
        [
            html.H5("Death Rate", className="card-title6", style={"text-align": "center"}),
            html.P(str(float("{:.2f}".format(np.sum(df['Total Deaths']) / np.sum(df['Confirmed Cases']) * 100)))+"%", 
            className="card-text6", style={"text-align": "center"})
        ]
    )
]

# Recovery rate
card_content_7 = [
    
    dbc.CardBody(
        [
            html.H5("Recovery Rate", className="card-title7", style={"text-align": "center"}),
            html.P(str(float("{:.2f}".format(np.sum(df['Recovered Cases']) / np.sum(df['Confirmed Cases']) * 100)))+"%", 
            className="card-text7", style={"text-align": "center"})
        ]
    )
]


In [9]:
app.layout = html.Div(
    [   
        # Header
        dbc.Row(dbc.Col(html.Div(html.H2("Covid-19 Dashboard",
                style={'textAlign':'center'}),style = {'margin-top' : '25px'}))),

        # Message
        dcc.ConfirmDialog(id='warning'),


        # BANS
        dbc.Row([
                # Confirmed
                dbc.Col(dbc.Card(card_content_1, color=pallete['Confirmed'], inverse=True), style = {'margin-left' : '12px'}),
                # Active
                dbc.Col(dbc.Card(card_content_2, color=pallete['Active'], inverse=True)),
                # Recovered
                dbc.Col(dbc.Card(card_content_5, color=pallete['Recovered'], inverse=True)),
                # Critical
                dbc.Col(dbc.Card(card_content_4, color=pallete['Critical'], inverse=True)),
                # Deaths
                dbc.Col(dbc.Card(card_content_3, color=pallete['Deaths'], inverse=True), style = {'margin-right' : '12px'}),
                # Recovery rate
                #dbc.Col(dbc.Card(card_content_7, color=pallete['Recovered'], inverse=True)),
                # Deaths rate
                #dbc.Col(dbc.Card(card_content_6, color=pallete['Deaths'], inverse=True), style = {'margin-right' : '12px'}),
            ],
            className="mb-4",  style={'margin-top':'25px'}
        ),

        
        dbc.Row(
            [
                # Confirmed Cases Map
                dbc.Col([
                    dbc.Row([
                            dbc.Col(html.Label(['Country'], style={'font-weight': 'bold',"text-align": "left", 'margin-left':'12px'}), width=1),
                            dbc.Col(dcc.Dropdown(id='myDropdown',
                                        options=[{'label': str(Country), 'value': str(Country)}
                                        for Country in df1['Country'].unique()], 
                                        value=None, multi=False
                            ), style={'margin-left':'50px'}, width=5)
                            ], style={'background':'#ffffff', 'margin':'0px', 'padding-top':'25px'}),

                    html.Div(dcc.Graph(id='Global_Map'))], width=6, style={'margin-left':"12px", 'padding-right':"0px"}
                ),
                
                
                # PiChart
                dbc.Col(html.Div(dcc.Graph(id='Chart')), width=4, 
                        style={'margin-left':'12px', 'padding':"0px", 'background':'#ffffff'}),
                
                # Side Cards
                dbc.Col(
                    [
                        dbc.Col(html.Div(dbc.Card([
                            dbc.CardBody(
                                [
                                    html.H5("Country With Highest Cases in This region", style={"padding-bottom":"20px"}),
                                    html.P(id="card_text_1", style={'color': pallete['Confirmed'], 'fontSize': 20}),
                                    html.P(id="card_text_2", style={'color': pallete['Active'], 'fontSize': 20}),
                                    html.P(id="card_text_3", style={'color': pallete['Recovered'], 'fontSize': 20}),
                                    html.P(id="card_text_4", style={'color': pallete['Critical'], 'fontSize': 20}),
                                    html.P(id="card_text_5", style={'color': pallete['Deaths'], 'fontSize': 20})
                                ], style={'height':'31.8rem'}
                            ),

                        ], color="#ffffff", outline=True)), style={"text-align": "center"}),

                    ], style={"margin-right" : "12px", 'margin-left':"12px", 'padding-left': '0px'}
                ),   
            ], style={'margin-top':'25px'}
        ),

        dbc.Col(html.Div(dcc.Graph(figure=Tests_Performed_fig)), style={'margin':'12px', 'margin-top':'25px'}),

        # Global vaccination trend (Cummulated)
        
        dbc.Row(
            [   
                dbc.Col(html.Div(dcc.Graph(figure=Total_Vaccinations_Trend_fig)), width=6,
                        style={'margin':'0px', 'margin-left':'24px', 'padding':'0px', 'background':'#ffffff'}),
                
                dbc.Col([
                        dbc.Row([
                            dbc.Col(dcc.RadioItems(options=['Countries', 'Continents'], value='Continents', id='selectOne_vac', labelStyle = {'display':'inline', 'margin-left':'12px'}), 
                                    width=4, style={"margin-left" : "12px"}), 

                            dbc.Col(dcc.Dropdown(multi=True, id='options_vac'))
                        ], style={'background':'#ffffff', 'margin':'0px', 'padding-top':'30px'}),

                        html.Div(dcc.Graph(id='vaccine_trend')
                            )],
                    style={'margin':"0px", 'margin-right':'12px'}
                )
                ], style={'margin-top':'25px'}
            ),

        # Total deaths vs recovered comparison
        
        dbc.Col([
            dbc.Row([
                dbc.Col(html.Label(['Country'], style={'font-weight': 'bold', 'margin-left':'24px', 'padding-right':'0px', 'margin-right':'0px'}), width=1),
                dbc.Col(dcc.Dropdown(options=comparison_countries, multi=True, value=['India', 'USA'], id='options_compare', style={"margin" : "0px", "padding-left" : "0px"}), width=6),
                dbc.Col(dbc.Button(children='Submit', id='submit', color="primary"), width=1)
            ], style={'margin-bottom':'5px'}),
            html.Div(dcc.Graph(id='compare_trend'), style={"margin" : "12px", 'margin-top':'0px', 'margin-bottom':'0px'})
        ], style={'background':'#ffffff', "margin" : "12px",  'margin-top':'25px','padding-top':'20px'}),   


        # Global Deaths   &&&   # Global confirmed cases   
        dbc.Row(
            [
                dbc.Col([
                    dbc.Row([
                    dbc.Col(dcc.RadioItems(options=['Countries', 'Continents'], value='Continents', labelStyle={'display':'inline', 'margin-left':'12px'}, id='selectOne_death')),
                    dbc.Col(dcc.Dropdown(multi=False, id='options_death'), width=8),
                    ], style={'background':'#ffffff', 'margin-bottom':'0px', "margin-right" : "0px", 'padding-top':'30px', "margin-left" : "12px"}),
                    dbc.Col(html.Div(dcc.Graph(id='death_trend')), style={"margin-top" : "0px", "margin-left" : "12px"}),
                ], style={'padding-right':'0px', 'margin-right':'6px'}),
                 
                 dbc.Col([
                    dbc.Row([
                    dbc.Col(dcc.RadioItems(options=['Countries', 'Continents'], value='Continents', id='selectOne_conf', labelStyle={'display':'inline', 'margin-left':'12px'})),    
                    dbc.Col(dcc.Dropdown(multi=False, id='options_conf'), width=8),
                    ], style={'background':'#ffffff', 'margin-bottom':'0px', 'margin-left':'0px', 'padding-top':'30px', "margin-right" : "12px"}),
                    dbc.Col(html.Div(dcc.Graph(id='conf_trend')), style={"margin-top" : "0px", "margin-right" : "12px"}),
                ], style={'padding-left':'0px', 'margin-left':'6px'})
            ], style={'margin-top':'25px', 'padding-bottom':'25px'}
        ), 
    ], style={'backgroundColor':pallete['background']}
)


## Callbacks

In [10]:
@app.callback(
    Output(component_id='Global_Map', component_property='figure'),
    Output(component_id='Chart', component_property='figure'),
    Output(component_id='card_text_1', component_property='children'),
    Output(component_id='card_text_2', component_property='children'),
    Output(component_id='card_text_3', component_property='children'),
    Output(component_id='card_text_4', component_property='children'),
    Output(component_id='card_text_5', component_property='children'),
    Input(component_id='myDropdown', component_property='value')
)

def update_my_fig(country):

    if country == None:
        map_filter = df1
        values = [np.sum(df2['Active Cases']), np.sum(df2['Recovered Cases']), np.sum(df2['Total Deaths'])]

        name1 = df[df['Confirmed Cases'] == np.max(df['Confirmed Cases'])].Country.values[0]
        num1 = "{:,}".format(int(np.max(df['Confirmed Cases'])))
        name2 = df[df['Active Cases'] == np.max(df['Active Cases'])].Country.values[0]
        num2 = "{:,}".format(int(np.max(df['Active Cases'])))
        name3 = df[df['Recovered Cases'] == np.max(df['Recovered Cases'])].Country.values[0]
        num3 = "{:,}".format(int(np.max(df['Recovered Cases'])))
        name4 = df[df['Critical Cases'] == np.max(df['Critical Cases'])].Country.values[0]
        num4 = "{:,}".format(int(np.max(df['Critical Cases'])))
        name5 = df[df['Total Deaths'] == np.max(df['Total Deaths'])].Country.values[0]
        num5 = "{:,}".format(int(np.max(df['Total Deaths'])))

    else:
        # map update
        map_filter = df1[np.isin(df1.Country,country)]

        # pie update
        pie_filter = df2[np.isin(df2.Country,country)]
        values = np.array(pie_filter.drop(['Country'], axis=1))
        values = list(chain.from_iterable(values))

        # cards update
        text_filter = df[np.isin(df.Country,country)].Continent.values[0]
        gk = df.groupby('Continent').get_group(text_filter)
        name1 = gk[gk['Confirmed Cases'] == np.max(gk['Confirmed Cases'])].Country.values[0]
        num1 = "{:,}".format(int(np.max(gk['Confirmed Cases'])))
        name2 = gk[gk['Active Cases'] == np.max(gk['Active Cases'])].Country.values[0]
        num2 = "{:,}".format(int(np.max(gk['Active Cases'])))
        name3 = gk[gk['Recovered Cases'] == np.max(gk['Recovered Cases'])].Country.values[0]
        num3 = "{:,}".format(int(np.max(gk['Recovered Cases'])))
        name4 = gk[gk['Critical Cases'] == np.max(gk['Critical Cases'])].Country.values[0]
        num4 = "{:,}".format(int(np.max(gk['Critical Cases'])))
        name5 = gk[gk['Total Deaths'] == np.max(gk['Total Deaths'])].Country.values[0]
        num5 = "{:,}".format(int(np.max(gk['Total Deaths'])))
        
    
    fig1 = px.choropleth(map_filter, locations="iso_alpha",
                    featureidkey="properties.iso_alpha",
                    projection="equirectangular",
                    color="Confirmed Cases", 
                    hover_name="Country",
                    hover_data=["Active Cases", "Total Deaths", "Recovered Cases"],
                    color_continuous_scale=px.colors.sequential.Reds)
    
    fig1.update_geos(fitbounds="locations", visible=True)
    
    fig1.update_layout(title_text='Confirmed Cases by Country', title_x=0.5, title_y=0.93, margin={"r":10,"t":0,"l":10,"b":0},
                        coloraxis_colorbar=dict(
                        title="Confirmed",
                        thicknessmode="pixels", thickness=10,
                        lenmode="pixels", len=150,
                        yanchor="bottom", y=0.14,
                        xanchor="left", x=0.01
                    )
                )

    labels = ['Active Cases', 'Recovered Cases', 'Total Deaths']
    fig2 = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.6)])
    fig2.update_layout(title_text="Overall Active Cases vs Recovered vs Deaths", title_x=0.5, margin=dict(t=60, b=0, l=0, r=0), legend=dict(orientation="h",
                                                                                                                                            yanchor="bottom",
                                                                                                                                            y=-0.1,
                                                                                                                                            xanchor="right",
                                                                                                                                            x=1)
                    )
    fig2.update_traces(marker=dict(colors=[pallete['Active'], pallete['Recovered'], pallete['Deaths']]))

    card1 = name1 + " : " + num1
    card2 = name2 + " : " + num2
    card3 = name3 + " : " + num3
    card4 = name4 + " : " + num4
    card5 = name5 + " : " + num5

    return fig1, fig2, card1, card2, card3, card4, card5

In [11]:
# Callbacks for comparison
@app.callback(
              Output(component_id='compare_trend', component_property='figure'),
              Output(component_id='options_compare', component_property='value'),
              Output(component_id='warning', component_property='displayed'),
              Output(component_id='warning', component_property='message'),
              Input(component_id='submit', component_property='n_clicks'),
              Input(component_id='compare_trend', component_property='figure'),
              State(component_id='options_compare', component_property='value')
              )

def update_graph_comparison(button_click, input_graph, dropdown_input):

    displayed = False
    message_text = ''
    
    dropdown_input = sorted(dropdown_input)
    if len(dropdown_input) > 5:
        
        # Prompt a message here
        displayed = True
        message_text = 'Maximum number of countries to be compared is 5!'
        
    
    if len(dropdown_input) < 2:
        
        # Prompt a message here
        displayed = True
        message_text = 'Minimum number of countries to be compared is 2!'
    
    if len(dropdown_input) >= 2 and len(dropdown_input) <= 5:
        
        confirmed_selected_countries = comparison_df[comparison_df['Country'].isin(dropdown_input)].sort_values(by='Country')
        # Total confirmed
        fig = go.Figure(go.Bar(x = dropdown_input,
                               y = confirmed_selected_countries['Total confirmed'], 
                               yaxis='y1', name='No. Total confirmed', width=0.5,
                               text=confirmed_selected_countries['Total confirmed'], textposition='outside',
                               hovertemplate=  "<br>Country: %{x}</b>" +
                                               "<br>Total Recovered: %{text}</br>"))



        # Total recovered
        recovered_scatter = go.Scatter(x=dropdown_input, y=confirmed_selected_countries['Total Recovered percent'], mode="markers",
                                       marker=dict(size=70 * confirmed_selected_countries['Total Recovered percent'], color=pallete['Recovered']),
                                       yaxis='y2', name='Total Recovered percentage', 
                                       text=confirmed_selected_countries['Total Recovered'], 
                                       hovertemplate=  "<br>Country: %{x}</b>" +
                                                    "<br>Total Recovered: %{text}</br>")

        recovered_text = [(x+"%") for x in (np.round(confirmed_selected_countries['Total Recovered percent']*100, 2)).astype(str).tolist()]
        recovered_text = [('<b>' + txt + '</b>') for txt in recovered_text]
        recovered_text_scatter = go.Scatter(x=dropdown_input, y=confirmed_selected_countries['Total Recovered percent'],
                                            text=recovered_text, mode='text', yaxis='y2', showlegend=False,
                                            textfont=dict(
                                            family="sans serif",
                                            size=12,
                                            color="#ffffff"
                                         ))

        fig.add_traces([recovered_scatter, recovered_text_scatter])


        # Total deaths
        deaths_scatter = go.Scatter(x=dropdown_input, y=confirmed_selected_countries['Total Deaths percent'], mode='lines+markers', 
                                    line_shape='spline', line=dict(width=1, color=pallete['Deaths']),
                                    marker=dict(size=5, color=pallete['Deaths']), 
                                    yaxis='y3', name='Total deaths percentage', 
                                    text=confirmed_selected_countries['Total Deaths'],
                                    hovertemplate=  "<br>Country: %{x}</b>" +
                                                    "<br>Total Deaths: %{text}</br>")

        deaths_text = [(x+"%") for x in (np.round(confirmed_selected_countries['Total Deaths percent']*100, 2)).astype(str).tolist()]
        deaths_text = [('<b>' + txt + '</b>') for txt in deaths_text]
        deaths_text_scatter = go.Scatter(x=dropdown_input, y=confirmed_selected_countries['Total Deaths percent'],
                                         text=deaths_text, mode='text', textposition='top left', yaxis='y3', showlegend=False,
                                         textfont=dict(
                                            family="sans serif",
                                            size=15,
                                            color=pallete['Deaths']
                                         ))

        fig.add_traces([deaths_scatter, deaths_text_scatter])



        # Create axis objects
        fig.update_layout(
            xaxis=dict(
                domain=[0.2, 0.9]
            ),
            yaxis=dict(
                title="Confirmed",
                titlefont=dict(
                    color=pallete['Confirmed']
                ),
                tickfont=dict(
                    color=pallete['Confirmed']
                )
            ),
            yaxis2=dict(
                title="Recovered",
                titlefont=dict(
                    color=pallete['Recovered']
                ),
                tickfont=dict(
                    color=pallete['Recovered']
                ),
                anchor="free",
                overlaying="y",
                side="left",
                position=0.05
            ),
            yaxis3=dict(
                title="Deaths",
                titlefont=dict(
                    color=pallete['Deaths']
                ),
                tickfont=dict(
                    color=pallete['Deaths']
                ),
                anchor="x",
                overlaying="y",
                side="right"
            )
        )

        return fig.update_layout(title_text="Total recovered vs total deaths country-wise"), dropdown_input, displayed, message_text
    
    else:
        return input_graph, dropdown_input, displayed, message_text

In [12]:
# Callbacks for global vaccination trend (Cummulated)
@app.callback(
              Output(component_id='options_vac', component_property='options'), 
              Output(component_id='options_vac', component_property='value'),
              Input(component_id='selectOne_vac', component_property='value')
              )

def update_dropdown_vaccine(radio_input):
    if radio_input == 'Continents':
        return list(set(continents)-set(['Middle East'])), None
    else:
        return Any_country, None


@app.callback(
              Output(component_id='vaccine_trend', component_property='figure'),
              Input(component_id='selectOne_vac', component_property='value'),
              Input(component_id='options_vac', component_property='value')
              )


def update_graph_vaccine(radio_input, dropdown_input):
    if dropdown_input == None:
        dropdown_input = []
        world = Vaccination_trend_Continents.groupby(by='date', as_index=False).sum()
        line_graph = px.line(world, x="date", y='total_vaccinations', markers=True).update_layout(title_text="World cummulated vaccination trend")
        
    elif radio_input == 'Continents':
        
        continent = Vaccination_trend_Continents[Vaccination_trend_Continents.location.isin(dropdown_input)]
        line_graph = px.line(continent, x="date", y='total_vaccinations', markers=True, color='location').update_layout(title_text="Continents cummulated vaccination trend", legend=dict(yanchor="top",
                                                                                                                                                                                          y=0.8,
                                                                                                                                                                                          xanchor="left",
                                                                                                                                                                                          x=0.01, 
                                                                                                                                                                                          bgcolor='rgba(255,255,255,0.6)'))
    
    else:

        country = Vaccination_trend_Countries[Vaccination_trend_Countries.location.isin(dropdown_input)]
        line_graph = px.line(country, x="date", y='total_vaccinations', markers=True, color='location').update_layout(title_text="Countries cummulated vaccination trend",legend=dict(yanchor="top",
                                                                                                                                                                                      y=0.8,
                                                                                                                                                                                      xanchor="left",
                                                                                                                                                                                      x=0.01, 
                                                                                                                                                                                      bgcolor='rgba(255,255,255,0.6)'))
        
    line_graph.update_traces(mode="markers+lines", hovertemplate=None)
    line_graph.update_layout(hovermode="x unified")

    return line_graph

In [13]:
# Callbacks for global Deaths 
@app.callback(
              Output(component_id='options_death', component_property='options'), 
              Output(component_id='options_death', component_property='value'),
              Input(component_id='selectOne_death', component_property='value')
              )


def update_dropdown_death(radio_input):
    if radio_input == 'Continents':
        return list(set(continents)-set(['Middle East'])), None
    else:
        return Any_country, None
    

@app.callback(
              Output(component_id='death_trend', component_property='figure'),
              Input(component_id='selectOne_death', component_property='value'),
              Input(component_id='options_death', component_property='value')
              )

def update_graph_death(radio_input, dropdown_input):
    if dropdown_input == None:
        dropdown_input = ""
        world = global_daily_trend_continents.groupby(by='date', as_index=False).sum()
        
        histo_graph = go.Figure(px.histogram(world, 
                             x='date', 
                             y='new_deaths', 
                             nbins=300, 
                             color_discrete_sequence=['lightgray'], 
                             hover_data=['date', 'new_deaths'], 
                             marginal='rug'))
        
        histo_graph.add_trace(go.Scatter(x=world['date'], y=world['new_deaths_smoothed'], name='World daily deaths trend line'))
    
    elif radio_input == 'Continents':
        
        continent = global_daily_trend_continents[global_daily_trend_continents.location == dropdown_input]
        histo_graph = go.Figure(px.histogram(continent, 
                                     x='date', 
                                     y='new_deaths', 
                                     nbins=150, 
                                     color_discrete_sequence=['lightgray'], 
                                     hover_data=['date', 'new_deaths'], 
                                     marginal='rug'))

        histo_graph.add_trace(go.Scatter(x=continent['date'], y=continent['new_deaths_smoothed'], name=f'{dropdown_input} daily deaths trend line'))
    
    else:

        country = global_daily_trend_countries[global_daily_trend_countries.location == dropdown_input]
        histo_graph = go.Figure(px.histogram(country, 
                                     x='date', 
                                     y='new_deaths', 
                                     nbins=100, 
                                     color_discrete_sequence=['lightgray'], 
                                     hover_data=['date', 'new_deaths'], 
                                     marginal='rug'))

        histo_graph.add_trace(go.Scatter(x=country['date'], y=country['new_deaths_smoothed'], name=f'{dropdown_input} daily deaths trend line'))
    
    
    return histo_graph.update_layout(title_text="Daily deaths", legend=dict(yanchor="top",
                                                                            y=0.8,
                                                                            xanchor="left",
                                                                            x=0.01, 
                                                                            bgcolor='rgba(255,255,255,0.6)', 
                                                                            title='Trend Line'))

In [14]:
# Callbacks for global confirmed cases
@app.callback(
              Output(component_id='options_conf', component_property='options'), 
              Output(component_id='options_conf', component_property='value'),
              Input(component_id='selectOne_conf', component_property='value')
              )

def update_dropdown_conf(radio_input):
    if radio_input == 'Continents':
        return list(set(continents)-set(['Middle East'])), None
    else:
        return Any_country, None
    

@app.callback(
              Output(component_id='conf_trend', component_property='figure'),
              Input(component_id='selectOne_conf', component_property='value'),
              Input(component_id='options_conf', component_property='value')
              )



def update_graph_conf(radio_input, dropdown_input):
    if dropdown_input == None:
        dropdown_input = ""
        world = global_daily_trend_continents.groupby(by='date', as_index=False).sum()
        
        confirmed_trend_line = go.Figure(go.Scatter(x=world['date'], y=world['new_cases'], name='World New Cases'))
        world_mean = np.average(world['new_cases'], axis=0)
        world_max = np.max(world['new_cases'], axis=0)
        confirmed_trend_line.update_traces(mode="markers+lines", hovertemplate=None)


        # Trend line
        confirmed_trend_line.add_trace(go.Scatter(x=world['date'], y=world['new_cases_smoothed'], name='Trend Line', line_width=1))
        
        confirmed_trend_line.add_hline(y=world_mean, line_width=1, line_color='green')
        confirmed_trend_line.add_annotation(text='Avg = {}'.format(int(world_mean)), x='2022-05-24', y=world_mean, arrowcolor="green", font=dict(family="Serif, monospace",
                                                                                                                                                 size=14,
                                                                                                                                                 color="green"))

        confirmed_trend_line.add_hline(y=world_max, line_width=1, line_color='green')
        confirmed_trend_line.add_annotation(text='Max = {}'.format(int(world_max)), x='2022-05-24', y=world_max, arrowcolor="green", font=dict(family="Serif, monospace",
                                                                                                                                                size=14,
                                                                                                                                                 color="green"))

    
    elif radio_input == 'Continents':
        continent = global_daily_trend_continents[global_daily_trend_continents.location == dropdown_input]
        confirmed_trend_line = go.Figure(go.Scatter(x=continent['date'], y=continent['new_cases'], name='New cases'))
        confirmed_trend_line.update_traces(mode="markers+lines", hovertemplate=None)

        continent_mean = np.average(continent['new_cases'], axis=0)
        continent_max = np.max(continent['new_cases'], axis=0)


        # Trend line
        confirmed_trend_line.add_trace(go.Scatter(x=continent['date'], y=continent['new_cases_smoothed'], name='Trend_line', line_width=1))
        confirmed_trend_line.add_hline(y=continent_mean, line_width=1, line_color='green')
        confirmed_trend_line.add_annotation(text='Avg = {}'.format(int(continent_mean)), x='2022-05-24', y=continent_mean, arrowcolor="green", font=dict(family="Serif, monospace",
                                                                                                                                                 size=14,
                                                                                                                                                 color="green"))

        confirmed_trend_line.add_hline(y=continent_max, line_width=1, line_color='green')
        confirmed_trend_line.add_annotation(text='Max = {}'.format(int(continent_max)), x='2022-05-24', y=continent_max, arrowcolor="green", font=dict(family="Serif, monospace",
                                                                                                                                                size=14,
                                                                                                                                                 color="green"))

        
    
    else:

        country = global_daily_trend_countries[global_daily_trend_countries.location == dropdown_input]
        confirmed_trend_line = go.Figure(go.Scatter(x=country['date'], y=country['new_cases'], name='New cases'))
        confirmed_trend_line.update_traces(mode="markers+lines", hovertemplate=None)
        country_mean = np.average(country['new_cases'], axis=0)
        country_max = np.max(country['new_cases'], axis=0)        
        
        # Trend line
        confirmed_trend_line.add_trace(go.Scatter(x=country['date'], y=country['new_cases_smoothed'], name='Trend_line', line_width=1))
        confirmed_trend_line.add_hline(y=country_mean, line_width=1, line_color='green')
        confirmed_trend_line.add_annotation(text='Avg = {}'.format(int(country_mean)), x='2022-05-24', y=country_mean, arrowcolor="green", font=dict(family="Serif, monospace",
                                                                                                                                                 size=14,
                                                                                                                                                 color="green"))

        confirmed_trend_line.add_hline(y=country_max, line_width=1, line_color='green')
        confirmed_trend_line.add_annotation(text='Max = {}'.format(int(country_max)), x='2022-05-24', y=country_max, arrowcolor="green", font=dict(family="Serif, monospace",
                                                                                                                                                size=14,
                                                                                                                                                 color="green"))

    
    return confirmed_trend_line.update_layout(title_text="Daily confirmed cases", hovermode="x unified", 
                                              legend=dict(yanchor="top",
                                                          y=0.85,
                                                          xanchor="left",
                                                          x=0.01, 
                                                          bgcolor='rgba(255,255,255,0.6)'))

## Run

In [15]:
app.run_server()

Dash app running on http://127.0.0.1:8050/
